In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib as plt
import numpy as np
import cv2

In [125]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg



img=mpimg.imread('IMG_1830.png')

def get_pt_onclick(event):
    global ix,iy, rgba
    ix, iy = event.xdata,event.ydata
    rgba = event.inaxes.get_images()[0].get_cursor_data(event).data
    print(ix, iy)



def cal_points(event): 
    fig.canvas.draw_idle()
    global ix,iy, calpts
    if len(calpts)<4:
        #print(event.key)
        if event.key == 'enter':
            calpts.append([ix,iy])
            plt.plot(ix,iy, 'rx')
#             if len(calpts)>=4:
#                 plt.close(plt.gcf())
                
def line_color(event):
    global ix,iy,rgba,line_col
    if event.key == '-':
        print(event.key)
        line_col = rgba 
        
def mouse_eraser(event):
    global ix,iy, pts
    if event.key == ' ':
        plt.plot(ix,iy, 'bx')

fig = plt.figure(figsize=(10,15))
cid = fig.canvas.mpl_connect('button_press_event', get_pt_onclick)
kid = fig.canvas.mpl_connect('key_press_event', cal_points)
lid = fig.canvas.mpl_connect('key_press_event', line_color)


In [126]:
ix,iy = 0,0
line_col = [0,0,0,0]
calpts = []
pts = 
imgplot = plt.imshow(img)
plt.gca().plot([300,300], 'ro')
plt.show()

In [91]:
calpts
# plt.gcf().canvas.mpl_connect('button_press_event', onclick)
# plt.gcf().canvas.mpl_connect('key_press_event', cal_points)

[[2235.689995421436, 1483.1808766867855],
 [3293.015494693029, 1475.7697166451621],
 [1924.421273673257, 1396.7173428678468],
 [1917.0101136316337, 705.0090723163378]]

In [120]:
mask = np.sum((img - np.tile(line_col,(img.shape[0],img.shape[1],1)))**2, axis = 2)

In [153]:
plt.figure(2)
plt.imshow(mask)
plt.colorbar()

In [154]:
mask2 = np.copy(mask)
mask2[mask2>0.05] = 1
plt.figure(3)
plt.imshow(mask2, cmap = 'inferno_r')
plt.colorbar()
plt.show()

In [158]:
pts = np.array(np.where((((X - xi)**2) < 1) & +(Y-yi)**2)

In [179]:
Y,X=np.meshgrid(np.arange(mask2.shape[0]),np.arange(mask2.shape[1]))

In [185]:
Y

array([[   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    1,    1,    1],
       [   2,    2,    2, ...,    2,    2,    2],
       ...,
       [4029, 4029, 4029, ..., 4029, 4029, 4029],
       [4030, 4030, 4030, ..., 4030, 4030, 4030],
       [4031, 4031, 4031, ..., 4031, 4031, 4031]])

In [102]:
line_col.data

array([0.52156866, 0.6117647 , 0.6901961 , 1.        ], dtype=float32)

In [155]:
plt.figure(2)
plt.plot(np.where(mask2<1)[1],np.where(mask2<1)[0], 'rx')
plt.show()

In [5]:
# Coordinates that you want to Perspective Transform
pts1 = np.float32([[0,0],\
                   [0,1024],\
                   [1024,0],\
                   [1024, 1024]])


# Size of the Transformed Image
pts2 = np.float32([[0+xshift + dd2,0 + yshift],\
                   [0+xshift,1024 + yshift],\
                   [1024+xshift + dd,0 + yshift],\
                   [1024+xshift,1024 + yshift]])

MM = cv2.getPerspectiveTransform(pts1,pts2)

Digitizer.ipynb  IMG_1830.HEIC    IMG_1830.png     README.md


In [14]:
img.shape

(3024, 4032, 4)